In [1]:
import pandas as pd
import json
from sklearn.cluster import KMeans
import numpy as np

In [ ]:
# file_path = '../item1_and_financial_variable/processed_data/df_stage4_5years.csv'
# df = pd.read_csv(file_path)
# df = df[df['Year']==2021]
# df[['CIK', 'Year', 'Embeddings']].to_csv("2021_embedding.csv", index=False)


In [2]:
file_path = 'baseline_data/2021_embedding.csv'
df = pd.read_csv(file_path)

# There are several firms that have more than one embedding, I only retain first one here
duplicates = df[df['CIK'].duplicated(keep=False)]
print(duplicates.sort_values(by="CIK"))
df = df.drop_duplicates(subset='CIK', keep='first')
df[df['CIK'].duplicated(keep=False)]

          CIK  Year                                         Embeddings
231    861459  2021  [0.005930817220360041, 0.010183613747358322, 0...
675    861459  2021  [0.005812405608594418, 0.0010421487968415022, ...
160    886128  2021  [0.0029625254683196545, -0.006495341192930937,...
196    886128  2021  [-0.00039266914245672524, -0.01439815759658813...
506    898437  2021  [-0.016832245513796806, 0.026567725464701653, ...
1158   898437  2021  [-0.016669191420078278, 0.02598797157406807, 0...
275    943819  2021  [-0.06954989582300186, -0.007926642894744873, ...
511    943819  2021  [0.0375671461224556, 0.035842668265104294, 0.0...
1086   943819  2021  [0.0047199055552482605, 0.02963181398808956, 0...
780   1590715  2021  [0.018716150894761086, 0.0005908565944992006, ...
927   1590715  2021  [0.0156477689743042, 0.005799402948468924, 0.0...


,CIK,Year,Embeddings


In [3]:
len(df)

1219

In [73]:
df_cluster = df

In [ ]:
# file_path = '../item1_and_financial_variable/compustat_entire_2005_2022.csv'
# df = pd.read_csv(file_path)
# df = df[['cik', 'fyear', 'tic']]
# df = df.rename(columns={'cik': 'CIK', 'fyear': 'Year'})
# df.to_csv("tic_mapping.csv", index=False)

# df.head()

In [74]:
file_path = "baseline_data/tic_mapping.csv"
df = pd.read_csv(file_path)
print(len(df))

# Several records in compustats have nan values
df = df.dropna(subset=['CIK', 'Year', 'tic'])
print(len(df))
print(df.head())
# merged_df = pd.merge(df, compustat, how='left', on=['CIK', 'Year'])


# Convert the data type
df['CIK'] = df["CIK"].astype("int64")
df['Year'] = df['Year'].astype("int64")

print(df_cluster['CIK'].dtype)
print(df['CIK'].dtype)

print(df_cluster['Year'].dtype)
print(df['Year'].dtype)

219761
167766
      CIK    Year  tic
0  1750.0  2004.0  AIR
1  1750.0  2005.0  AIR
2  1750.0  2006.0  AIR
3  1750.0  2007.0  AIR
4  1750.0  2008.0  AIR
int64
int64
int64
int64


In [75]:
# Get "tic" for each record. problem here is that several embeddings don't have matching tic.
merged_df = pd.merge(df_cluster, df, how='left', on=['CIK', 'Year'])
# print(merged_df[merged_df.isna().any(axis=1)])

# It might because some records from compustat have nan cik or fyear, and those records are deleted from the above cell
print(len(merged_df))
merged_df = merged_df.dropna()
print(len(merged_df))

# Some duplicate from compustats, droped those duplicated records
# print(merged_df[merged_df['CIK'].duplicated(keep=False)])
merged_df = merged_df.drop_duplicates(subset=["CIK", "Year"])

print(len(merged_df))
# print(merged_df.head())


1403
1381
1197


In [76]:

print(len(merged_df['tic'].unique()))

with open("baseline_data/firm_subset.json", "r") as file:
    subset = json.load(file)

merged_df = merged_df[merged_df['tic'].isin(subset)]
len(merged_df['tic'].unique())

1197


534

### Clustering without PCA

In [56]:
embedding_list = list(merged_df['Embeddings'].apply(json.loads))
len(embedding_list)

534

In [57]:
def cluster(target, n_clusters):
    # Convert to a numpy array
    X = np.array(target)

    # Initialize and fit the k-means model
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    kmeans.fit(X)

    # Get cluster assignments
    labels = kmeans.labels_
    print("Cluster Labels:", labels)

    # Get cluster centers
    centers = kmeans.cluster_centers_
    print("Cluster Centers:", centers)

    wcss = np.sum((X - centers[labels])**2)
    print("Within-Cluster Sum of Squares (WCSS):", wcss)

    return labels


In [59]:
labels_10 = cluster(embedding_list, 10)
merged_df['Cluster_10'] = labels_10
labels_100 = cluster(embedding_list, 100)
merged_df['Cluster_100'] = labels_100
merged_df.head()

Cluster Labels: [8 4 2 8 3 5 8 4 8 8 7 0 4 8 8 3 5 9 8 5 8 2 1 9 8 5 2 1 3 8 5 1 4 8 5 6 4
 8 1 0 7 1 2 2 9 8 4 8 9 7 0 8 5 4 0 5 9 4 5 9 9 3 5 9 1 1 3 5 9 0 3 5 9 4
 4 2 3 8 3 5 4 9 5 1 4 4 2 1 8 5 4 1 7 8 8 4 8 3 4 8 2 7 2 9 4 8 5 8 9 9 7
 4 9 7 8 4 5 8 7 2 9 8 7 1 1 7 7 5 1 0 1 4 6 8 3 7 7 6 8 5 4 4 5 0 1 9 7 3
 3 3 8 1 3 8 2 0 3 3 3 9 3 6 9 5 8 2 9 1 5 4 3 8 7 8 8 7 3 7 6 5 4 2 8 2 5
 8 8 2 5 4 4 2 5 4 9 2 7 8 8 1 0 9 7 9 5 1 1 5 5 1 4 9 8 1 1 4 7 8 7 0 7 4
 8 7 4 2 8 1 5 7 2 2 4 3 8 5 1 9 5 5 7 1 6 6 7 7 5 6 2 4 3 8 2 0 8 4 4 5 5
 9 1 9 0 5 2 4 1 3 5 7 4 8 7 9 8 4 3 1 8 1 4 9 4 8 2 2 2 9 2 2 4 7 1 4 1 9
 4 7 8 1 5 5 4 8 8 0 3 7 8 7 7 9 8 5 7 2 2 8 6 4 5 2 8 8 2 7 5 2 3 7 7 9 4
 8 8 2 2 9 7 3 8 4 1 3 4 3 7 5 0 4 4 8 3 8 5 6 4 6 4 7 7 7 3 0 5 9 4 9 8 3
 2 1 5 9 9 9 9 7 8 1 1 8 6 9 9 8 8 9 8 7 2 4 3 7 8 8 8 3 9 8 4 4 0 9 9 5 2
 5 1 1 7 4 5 8 2 5 8 7 8 8 4 8 5 2 3 7 7 9 8 3 6 8 4 8 2 1 3 8 4 4 2 9 5 3
 2 5 3 1 8 5 1 8 4 1 8 5 8 5 0 8 4 0 5 7 3 8 8 8 7 4 5 9 2 5 5 1 8 5 3 2 9
 8 8 5 0 

,CIK,Year,Embeddings,tic,Cluster_10,Cluster_100
2,1704715,2021,"[0.014074714854359627, 0.006938479840755463, 0...",AMR,8,72
3,1575515,2021,"[-0.023708730936050415, 0.01762891560792923, 0...",SFM,4,23
4,1125376,2021,"[0.04463111609220505, 0.0018805989529937506, 0...",ENSG,2,28
5,1674335,2021,"[-0.0054583558812737465, 0.04776754230260849, ...",JELD,8,19
6,1023128,2021,"[0.007417375687509775, -0.03694348409771919, -...",LAD,3,34


In [60]:
import yfinance as yf

def fetch_stock_data(df_full_set, year, mode):
    # Filter data for the given year
    df_processed = df_full_set[df_full_set.Year == year]

    # Get unique tickers
    tickers = df_processed['tic'].unique().tolist()

    # Define the period based on the mode
    if mode == 'in-sample':
        # In-sample mode: start and end within the same year
        start_date = f'{year}-01-01'
        end_date = f'{year}-12-31'
    elif mode == 'forward':
        # Forward mode: next two years
        start_date = f'{year+1}-01-01'
        end_date = f'{year+2}-12-31'
    elif mode == 'backtest':
        # Backtest mode: last 10 years, up until year-1
        start_date = f'{year-10}-01-01'
        end_date = f'{year-1}-12-31'
    else:
        raise ValueError("Mode must be 'in-sample', 'forward', or 'backtest'")

    # Fetch historical stock data for all tickers
    stock_data = yf.download(tickers, threads= False, start=start_date, end=end_date)

    # Get adjusted close prices
    close_prices = stock_data['Adj Close']

    # Drop columns where all values are NaN
    close_prices = close_prices.dropna(axis=1, how='all')

    # Calculate daily returns
    returns = close_prices.pct_change()

    # Melt the returns DataFrame to long format for merging
    returns_long = returns.reset_index().melt(id_vars='Date', var_name='tic', value_name='Return')



# Merge the returns with the original dataframe



    return returns_long


In [61]:
#Start of where Tongzhou should reference

year = 2021
mode = 'forward'  # Can be 'in-sample', 'forward', or 'backtest'
# returns_long = fetch_stock_data(merged_df, year, mode)
returns_long = pd.read_csv("baseline_data/return_for_merged_item1.csv")

df_returns = pd.merge(merged_df, returns_long, on='tic', how='inner')


In [62]:
print(len(df_returns['tic'].unique()))

with open("baseline_data/firm_subset.json", "r") as file:
    subset = json.load(file)

df_returns = df_returns[df_returns['tic'].isin(subset)]
len(df_returns['tic'].unique())


534


534

In [18]:
def intra_industry_correlations(df, classification_column):
    intra_matrix = []
    industry_groups = df.groupby(classification_column)
    intra_correlations = {}
    for industry, group in industry_groups:
        pivot_table = group.pivot(index='Date', columns='tic', values='Return')
        corr_matrix = pivot_table.corr()
        # Take the upper triangle of the correlation matrix without the diagonal
        intra_corr = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool)).stack().mean()
        intra_correlations[industry] = intra_corr
    return intra_correlations

In [63]:
intra_corrs_10 = intra_industry_correlations(df_returns, "Cluster_10")
avg_intra_corr_10 = np.nanmean(list(intra_corrs_10.values()))
avg_intra_corr_10

0.3472796232527919

In [64]:
intra_corrs_100 = intra_industry_correlations(df_returns, "Cluster_100")
avg_intra_corr_100 = np.nanmean(list(intra_corrs_100.values()))
avg_intra_corr_100

0.4172189683301795

In [ ]:
# merged_df.to_csv("merged_df_from_item1.csv", index=False)

### PCA dimension reduction

In [77]:
file_path = "baseline_data/merged_df_from_item1.csv"
item1_df = pd.read_csv(file_path)
item1_df = item1_df.drop(['Cluster_10', 'Cluster_100'], axis=1)
item1_df.head()

print(len(item1_df['tic'].unique()))

with open("baseline_data/firm_subset.json", "r") as file:
    subset = json.load(file)

item1_df = item1_df[item1_df['tic'].isin(subset)]
len(item1_df['tic'].unique())

1197


534

In [78]:
from sklearn.decomposition import PCA
embedding_list = list(item1_df['Embeddings'].apply(json.loads))

pca = PCA(n_components=10)
embedding_pca = pca.fit_transform(embedding_list)
embedding_pca = np.array(embedding_pca)
len(embedding_pca)

534

In [79]:
labels_10 = cluster(embedding_pca, 10)
item1_df['Cluster_10'] = labels_10
labels_100 = cluster(embedding_pca, 100)
item1_df['Cluster_100'] = labels_100
item1_df.head()

Cluster Labels: [0 6 2 1 9 5 8 6 1 8 4 0 6 8 0 9 5 5 8 7 8 2 3 5 8 7 2 6 9 8 7 4 6 0 7 1 6
 0 4 0 0 5 2 2 5 8 6 8 5 3 1 8 1 6 0 7 5 6 5 5 5 1 1 5 4 4 9 7 5 0 9 7 4 6
 6 2 9 0 9 0 6 4 7 4 6 6 2 4 8 7 6 4 0 8 8 6 8 9 6 8 2 0 2 5 1 1 7 0 4 5 0
 6 5 0 8 8 7 8 0 2 5 1 3 4 4 5 3 7 4 0 4 6 4 8 9 3 3 4 0 7 6 1 7 0 4 5 3 9
 3 9 8 5 9 0 2 0 9 9 9 5 9 4 5 7 8 2 5 4 7 6 9 8 1 8 8 0 9 6 4 1 1 2 8 2 7
 8 0 2 7 6 6 2 1 6 5 2 1 8 1 4 0 5 3 5 5 4 4 1 7 4 6 5 1 1 1 6 3 8 1 1 3 6
 1 1 6 2 1 4 7 3 2 2 8 9 0 7 4 5 7 7 1 4 4 4 3 1 1 5 2 6 3 8 2 0 0 6 6 7 7
 5 6 5 0 7 2 6 6 9 7 3 6 1 3 5 0 6 9 4 0 4 6 4 6 8 2 2 2 5 2 2 6 1 4 6 4 4
 6 1 0 4 7 7 6 8 0 0 9 3 8 3 1 4 8 9 3 2 2 1 4 6 7 2 1 8 2 3 7 2 1 3 0 5 6
 8 8 2 2 5 1 9 0 6 4 9 6 9 9 7 0 6 6 1 9 8 7 4 6 4 6 0 9 0 9 0 7 5 8 5 8 9
 2 4 5 5 5 5 5 0 1 4 4 8 4 5 5 8 0 4 0 3 2 6 9 1 1 8 8 9 5 8 6 6 0 5 5 5 2
 5 4 6 3 6 5 8 2 5 8 0 8 8 6 8 7 2 9 3 1 4 1 9 5 8 6 8 2 4 9 1 6 6 2 5 7 9
 2 7 9 4 8 7 1 8 1 4 0 9 0 7 0 8 6 0 5 1 1 8 8 8 3 6 7 5 2 5 7 4 0 3 9 2 4
 0 8 7 0 

,CIK,Year,Embeddings,tic,Cluster_10,Cluster_100
2,1704715,2021,"[0.014074714854359627, 0.006938479840755463, 0...",AMR,0,25
3,1575515,2021,"[-0.023708730936050415, 0.01762891560792923, 0...",SFM,6,30
4,1125376,2021,"[0.04463111609220505, 0.0018805989529937506, 0...",ENSG,2,12
5,1674335,2021,"[-0.0054583558812737465, 0.04776754230260849, ...",JELD,1,91
6,1023128,2021,"[0.007417375687509775, -0.03694348409771919, -...",LAD,9,22


In [ ]:
year = 2021
mode = 'forward'  # Can be 'in-sample', 'forward', or 'backtest'
# returns_long = fetch_stock_data(item1_df, year, mode)

df_returns = pd.merge(item1_df, returns_long, on='tic', how='inner')

In [81]:
print(len(df_returns['tic'].unique()))

534


In [ ]:
# # returns_long.to_csv("baseline_data/return_for_merged_item1.csv", index=False)
# returns_long = pd.read_csv("baseline_data/return_for_merged_item1.csv")
# df_returns = pd.merge(item1_df, returns_long, on='tic', how='inner')

In [82]:
intra_corrs_10 = intra_industry_correlations(df_returns, "Cluster_10")
avg_intra_corr_10 = np.nanmean(list(intra_corrs_10.values()))
avg_intra_corr_10

0.3466661552205877

In [83]:
intra_corrs_100 = intra_industry_correlations(df_returns, "Cluster_100")
avg_intra_corr_100 = np.nanmean(list(intra_corrs_100.values()))
avg_intra_corr_100

0.3959588777109477

In [49]:
df_returns['Cluster_1'] = np.ones(len(df_returns), dtype="int64")
df_returns.head()

,CIK,Year,Embeddings,tic,Cluster_10,Cluster_100,Date,Return,Cluster_1
0,1704715,2021,"[0.014074714854359627, 0.006938479840755463, 0...",AMR,0,26,2022-01-03,NaN,1
1,1704715,2021,"[0.014074714854359627, 0.006938479840755463, 0...",AMR,0,26,2022-01-04,0.016987,1
2,1704715,2021,"[0.014074714854359627, 0.006938479840755463, 0...",AMR,0,26,2022-01-05,-0.019981,1
3,1704715,2021,"[0.014074714854359627, 0.006938479840755463, 0...",AMR,0,26,2022-01-06,0.001434,1
4,1704715,2021,"[0.014074714854359627, 0.006938479840755463, 0...",AMR,0,26,2022-01-07,-0.002704,1


In [50]:
intra_corrs_1 = intra_industry_correlations(df_returns, "Cluster_1")
avg_intra_corr_1 = np.nanmean(list(intra_corrs_1.values()))
avg_intra_corr_1

0.30203190501983